<a href="https://colab.research.google.com/github/FranMoretti/FranMoretti/blob/main/Copia_de_05_lightfm.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
! pip install lightfm > /dev/null
# necesita python 3.10

In [2]:
import lightfm as lfm
from lightfm import data
from lightfm import cross_validation
from lightfm import evaluation
import pandas as pd
import numpy as np
import sqlite3

In [111]:
conn = sqlite3.connect('data.db')

df_total = pd.read_sql_query("""SELECT i.id_libro, l.genero, AVG(i.rating) AS promedio_rating, COUNT(i.rating) AS interacciones
FROM interacciones AS i JOIN libros AS l ON i.id_libro = l.id_libro
GROUP BY i.id_libro
HAVING COUNT(*) >= 350
ORDER BY promedio_rating DESC, interacciones DESC
LIMIT 10
""" , conn) #  WHERE rating >= 7
#EL WHERE LO PONE PORQUE ESTE MODELO LE DA LO MISMO UN 1 QUE UN 10, A AMBOS LE ASIGNA 1, ASIGNA CERO A LOS QUE NO TIENEN INTERACCIONES
#ESTE MODELO REEMPLAZA SURPRISE
#ES UNA ESPECIE DE FILTRADO COLABORATIVO
#puedo usar este modelo y guardarlo en pickle por ejemplo
df_total.size
df_total

,id_libro,genero,promedio_rating,interacciones
0,las-legiones-malditas,Histórica y aventuras,8.879346,489
1,tormenta-de-espadas-cancion-de-hielo-y-fuego-3,"Fantástica, ciencia ficción",8.604762,630
2,los-miserables,Clásicos de la literatura,8.570888,529
3,africanus-el-hijo-del-consul,Histórica y aventuras,8.513889,432
4,mil-soles-esplendidos,Ficción literaria,8.503807,788
5,patria,Narrativa,8.470509,746
6,juego-de-tronos-cancion-de-hielo-y-fuego-1,"Fantástica, ciencia ficción",8.461272,865
7,el-padrino,"Novela negra, intriga, terror",8.431461,445
8,el-conde-de-montecristo,Histórica y aventuras,8.417349,611
9,cometas-en-el-cielo,Ficción literaria,8.319948,772


In [73]:
df_total.()

,id_libro,promedio_rating,interacciones
80,el-senor-de-las-moscas,7.262264,530
81,los-miserables,8.570888,529
82,sinsajo-los-juegos-del-hambre-3,7.177481,524
83,el-viejo-y-el-mar,7.213052,521
84,la-isla-del-tesoro,7.771372,503


In [6]:
conn = sqlite3.connect('data.db')

df_int = pd.read_sql_query("SELECT * FROM interacciones", conn) #  WHERE rating >= 7
#EL WHERE LO PONE PORQUE ESTE MODELO LE DA LO MISMO UN 1 QUE UN 10, A AMBOS LE ASIGNA 1, ASIGNA CERO A LOS QUE NO TIENEN INTERACCIONES
#ESTE MODELO REEMPLAZA SURPRISE
#ES UNA ESPECIE DE FILTRADO COLABORATIVO
#puedo usar este modelo y guardarlo en pickle por ejemplo
df_int.head()

,id_lector,id_libro,fecha,rating
0,yura,-incluso-el-olvido,hace 8 años,6
1,albertofg1,1080-recetas-de-cocina,hace 4 años,9
2,trini,1080-recetas-de-cocina,hace 12 años,10
3,chicarela,11-4-suenos-luz,hace 4 años,9
4,jorge-knowles-5,13-99-euros,hace 10 años,9


In [14]:
ds = lfm.data.Dataset()
ds.fit(users=df_int["id_lector"].unique(), items=df_int["id_libro"].unique())
ds.interactions_shape()

(11221, 44460)

In [15]:
(interactions, weights) = ds.build_interactions(df_int[['id_lector', 'id_libro', 'rating']].itertuples(index=False)) # ojo con el orden
#interactions
#weights

In [ ]:
#PRUEBA NO EXITOSA!!!!!!!!!!!!!!!!
from scipy.sparse import csr_matrix, coo_matrix
# Construir interacciones
(interactions, _) = ds.build_interactions(
    df_int[['id_lector', 'id_libro', 'rating']].itertuples(index=False)
)

row, col, _ = interactions.row, interactions.col, interactions.data
weights = np.where(df_int['rating'] >= 7, 2.0, 1.0)

# Crear matriz dispersa para los pesos en formato COO
weights_coo = coo_matrix((weights, (row, col)), shape=interactions.shape)

# Dividir en conjuntos de entrenamiento y prueba
train, test = lfm.cross_validation.random_train_test_split(interactions, test_percentage=0.2, random_state=42)
train_w, test_w = lfm.cross_validation.random_train_test_split(weights_coo, test_percentage=0.2, random_state=42)

# Asegurarse de que train_w esté alineado con train
train_w = coo_matrix(train_w)

# Crear y entrenar el modelo
model = lfm.LightFM(no_components=40, loss="bpr", learning_rate=0.01)
model.fit(train, sample_weight=train_w, epochs=10, num_threads=2)

# Evaluar precisión
train_precision = lfm.evaluation.precision_at_k(model, train, k=10, num_threads=2).mean()
test_precision = lfm.evaluation.precision_at_k(model, test, k=10, num_threads=2).mean()

print('Precisión en train: {}'.format(train_precision))
print('Precisión en test: {}'.format(test_precision))

Precisión en train: 0.09685785323381424
Precisión en test: 0.03339698165655136


In [16]:
train, test = lfm.cross_validation.random_train_test_split(interactions, test_percentage=0.2, random_state=42)
train_w, test_w = lfm.cross_validation.random_train_test_split(weights, test_percentage=0.2, random_state=42)
model = lfm.LightFM(no_components=30, loss="bpr",learning_rate = 0.02) # hiperparámetro
model.fit(train, sample_weight=train_w, epochs=10, num_threads=2)

In [17]:
train_precision = lfm.evaluation.precision_at_k(model, train, k=10, num_threads=2).mean()
test_precision  = lfm.evaluation.precision_at_k(model, test,  k=10, num_threads=2).mean()
print('Precision en train: {}'.format(train_precision))
print('Precision en test: {}'.format(test_precision))

Precision en train: 0.17784658074378967
Precision en test: 0.05117141827940941


In [ ]:
#AGREGADO POR MI PARA PROBARRRRR!!!!!!!!!!!!!!!!
# Probar diferentes hiperparámetros
model_configs = [
    {'no_components': 20, 'loss': 'bpr', 'learning_rate': 0.02},
    {'no_components': 30, 'loss': 'bpr', 'learning_rate': 0.02},
    {'no_components': 40, 'loss': 'bpr', 'learning_rate': 0.02}
]

# Asumo que ya tienes ds, interactions y weights definidos de tu código anterior
for config in model_configs:
    # Crear y entrenar el modelo con la configuración actual
    model = lfm.LightFM(**config)
    model.fit(train, sample_weight=train_w, epochs=10, num_threads=2)

    # Calcular métricas para esta configuración
    train_precision = evaluation.precision_at_k(model, train, k=10, num_threads=2).mean()
    test_precision = evaluation.precision_at_k(model, test, k=10, num_threads=2).mean()

    print(f"Configuración: {config}")
    print(f'Precision en train: {train_precision}')
    print(f'Precision en test: {test_precision}')

Configuración: {'no_components': 20, 'loss': 'bpr', 'learning_rate': 0.02}
Precision en train: 0.16185224056243896
Precision en test: 0.04936956241726875
Configuración: {'no_components': 30, 'loss': 'bpr', 'learning_rate': 0.02}
Precision en train: 0.1761338710784912
Precision en test: 0.05198138952255249
Configuración: {'no_components': 40, 'loss': 'bpr', 'learning_rate': 0.02}
Precision en train: 0.17375677824020386
Precision en test: 0.05042029917240143


In [ ]:
#AGREGADO POR MI PARA PROBARRRRR!!!!!!!!!!!!!!!!
# Early stopping
from lightfm.evaluation import precision_at_k

def train_with_early_stopping(model, train, test, train_w, test_w, epochs=50, patience=5, num_threads=2):
    best_precision = 0
    epochs_without_improvement = 0

    for epoch in range(epochs):
        model.fit(train, sample_weight=train_w, epochs=1, num_threads=num_threads)

        # Evaluar en conjunto de prueba
        test_precision = precision_at_k(model, test, k=10, num_threads=num_threads).mean()
        print(f"Época {epoch + 1}: Precisión en test: {test_precision:.4f}")

        # Verificar si hay mejora
        if test_precision > best_precision:
            best_precision = test_precision
            epochs_without_improvement = 0
        else:
            epochs_without_improvement += 1

        # Early stopping
        if epochs_without_improvement >= patience:
            print(f"Detenido en la época {epoch + 1} debido a falta de mejora.")
            break

    return model

# Llamar a la función de entrenamiento
model = lfm.LightFM(no_components=40, loss="bpr", learning_rate=0.02)
model = train_with_early_stopping(model, train, test, train_w, test_w, epochs=50, patience=5, num_threads=2)


Época 1: Precisión en test: 0.0398
Época 2: Precisión en test: 0.0417
Época 3: Precisión en test: 0.0418
Época 4: Precisión en test: 0.0395
Época 5: Precisión en test: 0.0385
Época 6: Precisión en test: 0.0412
Época 7: Precisión en test: 0.0409
Época 8: Precisión en test: 0.0405
Detenido en la época 8 debido a falta de mejora.


# Item features

In [29]:
conn = sqlite3.connect('data.db')

df_items = pd.read_sql_query("SELECT * FROM libros", conn)
df_items = df_items[["id_libro", "autor", "genero"]]
df_items.head()

,id_libro,autor,genero
0,las-particulas-elementales,"HOUELLEBECQ, MICHEL",Narrativa
1,quien-domina-el-mundo,"CHOMSKY, NOAM",Ensayo
2,antimanual-de-filosofia,"ONFRAY, MICHEL",Ensayo
3,momentos-estelares-de-la-humanidad-catorce-min...,"ZWEIG, STEFAN",Histórica y aventuras
4,la-invencion-del-pasado,"MURADO, MIGUEL-ANXO",Lecturas complementarias


In [30]:
ds = lfm.data.Dataset()
ds.fit(users=df_int["id_lector"].unique(), items=df_items["id_libro"].unique(), item_features=df_items["autor"].unique().tolist() + df_items["genero"].unique().tolist())
ds.interactions_shape()

(11221, 44670)

In [31]:
#df_items["autor"].unique().tolist() + df_items["genero"].unique().tolist()
len(df_items["autor"].unique().tolist() + df_items["genero"].unique().tolist())

19222

In [32]:
ifs = []
for _, row in df_items.iterrows():
    ifs.append( (row["id_libro"], [row["autor"], row["genero"]]) ) # { row["autor"]: 0.2, row["genero"]: 0.8 }

item_features = ds.build_item_features(ifs)
item_features

<44670x63891 sparse matrix of type '<class 'numpy.float32'>'
	with 134009 stored elements in Compressed Sparse Row format>

In [33]:
len(ifs)
24306

(interactions, weights) = ds.build_interactions(df_int[['id_lector', 'id_libro', 'rating']].itertuples(index=False, name=None)) # ojo con el orden
#interactions
#weights

In [48]:
train, test = lfm.cross_validation.random_train_test_split(interactions, test_percentage=0.2, random_state=42)
train_w, test_w = lfm.cross_validation.random_train_test_split(weights, test_percentage=0.2, random_state=42)
model = lfm.LightFM(no_components=30, loss="warp")
model.fit(train, sample_weight=train_w, epochs=15, item_features=item_features, num_threads=2)

In [49]:
train_precision = lfm.evaluation.precision_at_k(model, train, item_features=item_features, k=10, num_threads=2).mean()
test_precision  = lfm.evaluation.precision_at_k(model, test,  item_features=item_features, k=10, num_threads=2).mean()
print('Precision en train: {}'.format(train_precision))
print('Precision en test: {}'.format(test_precision))

Precision en train: 0.16168652474880219
Precision en test: 0.05567912384867668


# Predicción

In [18]:
# consigo los mapeos creados por el modelo // el modelo a cada libro/user le crea un ID
user_id_mapping, user_feature_mapping, item_id_mapping, item_feature_mapping = ds.mapping()
user_id_mapping
#item_id_mapping

{'yura': 0,
 'albertofg1': 1,
 'trini': 2,
 'chicarela': 3,
 'jorge-knowles-5': 4,
 'laconpe': 5,
 'dandreru': 6,
 'txuma': 7,
 'mari-cruz-vazquez': 8,
 'totote': 9,
 'muxia': 10,
 'gisela33': 11,
 'aketza': 12,
 'sebastian-almeida-ca-ez': 13,
 'zetarules': 14,
 'manelgs': 15,
 'elmullir': 16,
 'libros-delmundo-5': 17,
 'bender78': 18,
 'jose-antonio-lopez-martinez': 19,
 'florinata': 20,
 'paubaston': 21,
 'nupariona': 22,
 'emailpjc': 23,
 'smassip': 24,
 'alvarovalencia1219atgmailcom': 25,
 'anarodon': 26,
 'julian03': 27,
 'miguel-marchena': 28,
 'tonysukar': 29,
 'humilde-lector': 30,
 'auri3': 31,
 'lalu': 32,
 'josemariagainza': 33,
 'delarrubia': 34,
 'albertopetar': 35,
 'teckdive': 36,
 'danielprat': 37,
 'luis-2015': 38,
 'eharris1979': 39,
 'grinn': 40,
 'sintraholaatgmailcom': 41,
 'zaratrusta95': 42,
 'jere95': 43,
 'aleduardo': 44,
 'busta': 45,
 'vowel': 46,
 'doctorflorpo': 47,
 'bobina': 48,
 'sirandoni': 49,
 'razor95': 50,
 'wsmith': 51,
 '1783952111666490': 52,
 'a

In [19]:
model = lfm.LightFM(no_components=20)
model.fit(interactions=interactions, sample_weight=weights, item_features=item_features, epochs=10, num_threads=2)

In [26]:
#ESTA ES UNA PRUEBA CON UN USER Y DOS LIBROS, PERO YO DEBERÍA PASARLE TODOS LOS LIBROS QUE NO LEYÓ
#ESTE ES UNA PREDICCIÓN DE "ESTRELLITAS"
#ESTE ES UN MÉTODO MÁS RAPIDO
ID_LECTOR = "c3po"
IDS_LIBROS = ["yerma", "watchmen","la-caida-de-los-gigantes-the-century-1","la-ladrona-de-libros","un-mundo-sin-fin-los-pilares-de-la-tierra-2","la-catedral-del-mar","el-ultimo-caton"] # todos los items

#user_id_mapping[ID_LECTOR]
#np.array([item_id_mapping[m] for m in IDS_LIBROS])
model.predict(user_id_mapping[ID_LECTOR], np.array([item_id_mapping[m] for m in IDS_LIBROS]), item_features=item_features, num_threads=2)

array([13.970118, 14.906748, 11.628887, 14.744257, 15.183885, 14.582722,
       15.153565, 14.815198], dtype=float32)

In [21]:
# PARA VER
#ESTA ES UNA PREDICCIÓN DE RANKING // ESTE ES UN MÉTODO QUE SIRVE MÁS PARA POCOS USERS, PERO MAS LENTO SI HACES MUCHAS PREDICCIONES
model.predict_rank(interactions, item_features=item_features, num_threads=2, check_intersections=True)

<11221x44460 sparse matrix of type '<class 'numpy.float32'>'
	with 402601 stored elements in Compressed Sparse Row format>

In [ ]:
# https://making.lyst.com/lightfm/docs/lightfm.html